<a href="https://colab.research.google.com/github/Allen123321/DEMO-DL/blob/master/30_days_deep_learning_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive  #挂载drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip -o /content/Test.csv.zip # linux 解压命令

Archive:  /content/Test.csv.zip
  inflating: Test.csv                


In [ ]:

!unzip -o /content/Valid.csv.zip

Archive:  /content/Train.csv.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/Train.csv.zip or
        /content/Train.csv.zip.zip, and cannot find /content/Train.csv.zip.ZIP, period.
Archive:  /content/Valid.csv.zip
  inflating: Valid.csv               


In [ ]:
!unzip -o /content/Train.csv.zip

Archive:  /content/Train.csv.zip
  inflating: Train.csv               


example-1

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras import models,layers,preprocessing,optimizers,losses,metrics,datasets
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import re,string
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM



In [ ]:
(train_data,train_labels),(test_data,test_labels) = imdb.load_data(num_words=10000)


# train_data_path = "/content/Train.csv"
# test_data_path =  "/content/Test.csv"

MAX_WORDS = 10000  # 仅考虑最高频的10000个词
MAX_LEN = 200  # 每个样本保留200个词的长度
BATCH_SIZE = 20 

In [ ]:
train_data=sequence.pad_sequences(train_data, maxlen = MAX_LEN)
test_data=sequence.pad_sequences(test_data, maxlen = MAX_LEN)

In [ ]:
embedding_vector_length=32
model=Sequential()
model.add(layers.Embedding(MAX_WORDS,embedding_vector_length,input_length=MAX_LEN))
model.add(LSTM(100)) # LSTM的具体用法参考 https://keras.io/layers/recurrent/#lstm
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 200, 32)           320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 373,301
Trainable params: 373,301
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(train_data,train_labels,validation_data=(test_data,test_labels),epochs=5,batch_size=64)

Epoch 1/5
391/391 [==============================] - 115s 294ms/step - loss: 0.4191 - accuracy: 0.8030 - val_loss: 0.3793 - val_accuracy: 0.8440
Epoch 2/5
391/391 [==============================] - 116s 296ms/step - loss: 0.2440 - accuracy: 0.9075 - val_loss: 0.4330 - val_accuracy: 0.8279
Epoch 3/5
391/391 [==============================] - 116s 298ms/step - loss: 0.1858 - accuracy: 0.9322 - val_loss: 0.3457 - val_accuracy: 0.8616
Epoch 4/5
391/391 [==============================] - 116s 298ms/step - loss: 0.1499 - accuracy: 0.9464 - val_loss: 0.3954 - val_accuracy: 0.8445
Epoch 5/5
391/391 [==============================] - 115s 295ms/step - loss: 0.1144 - accuracy: 0.9595 - val_loss: 0.4001 - val_accuracy: 0.8604


example - 2

文本数据建模流程范例

imdb数据集的目标是根据电影评论的文本内容预测评论的情感标签。

训练集有20000条电影评论文本，测试集有5000条电影评论文本，其中正面评论和负面评论都各占一半。

文本数据预处理较为繁琐，包括中文切词（本示例不涉及），构建词典，编码转换，序列填充，构建数据管道等等。

在tensorflow中完成文本数据预处理的常用方案有两种，第一种是利用tf.keras.preprocessing中的Tokenizer词典构建工具和tf.keras.utils.Sequence构建文本数据生成器管道。

第二种是使用tf.data.Dataset搭配.keras.layers.experimental.preprocessing.TextVectorization预处理层。

In [30]:
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras import models,layers,preprocessing,optimizers,losses,metrics
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import re,string

train_data_path = "/content/train.csv"
test_data_path =  "/content/test.csv"

MAX_WORDS = 10000  # 仅考虑最高频的10000个词
MAX_LEN = 200  # 每个样本保留200个词的长度
BATCH_SIZE = 20 


#构建管道
def split_line(line):
    arr = tf.strings.split(line,"\t")
    label = tf.expand_dims(tf.cast(tf.strings.to_number(arr[0]),tf.int32),axis = 0)
    text = tf.expand_dims(arr[1],axis = 0)
    return (text,label)

ds_train_raw =  tf.data.TextLineDataset(filenames = [train_data_path]) \
   .map(split_line,num_parallel_calls = tf.data.experimental.AUTOTUNE) \
   .shuffle(buffer_size = 1000).batch(BATCH_SIZE) \
   .prefetch(tf.data.experimental.AUTOTUNE)

ds_test_raw = tf.data.TextLineDataset(filenames = [test_data_path]) \
   .map(split_line,num_parallel_calls = tf.data.experimental.AUTOTUNE) \
   .batch(BATCH_SIZE) \
   .prefetch(tf.data.experimental.AUTOTUNE)


#构建词典
def clean_text(text):
    lowercase = tf.strings.lower(text)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    cleaned_punctuation = tf.strings.regex_replace(stripped_html,
         '[%s]' % re.escape(string.punctuation),'')
    return cleaned_punctuation

vectorize_layer = TextVectorization(
    standardize=clean_text,
    split = 'whitespace',
    max_tokens=MAX_WORDS-1, #有一个留给占位符
    output_mode='int',
    output_sequence_length=MAX_LEN)

ds_text = ds_train_raw.map(lambda text,label: text)
vectorize_layer.adapt(ds_text)
print(vectorize_layer.get_vocabulary()[0:100])


#单词编码
ds_train = ds_train_raw.map(lambda text,label:(vectorize_layer(text),label)) \
    .prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test_raw.map(lambda text,label:(vectorize_layer(text),label)) \
    .prefetch(tf.data.experimental.AUTOTUNE)

['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i', 'this', 'that', 'was', 'as', 'for', 'with', 'movie', 'but', 'film', 'on', 'you', 'not', 'are', 'his', 'have', 'be', 'he', 'one', 'its', 'all', 'at', 'by', 'an', 'they', 'from', 'who', 'so', 'like', 'her', 'or', 'just', 'about', 'has', 'if', 'out', 'some', 'there', 'what', 'good', 'when', 'more', 'very', 'even', 'no', 'would', 'she', 'my', 'up', 'which', 'time', 'story', 'only', 'their', 'really', 'had', 'were', 'can', 'see', 'me', 'we', 'than', 'well', 'much', 'get', 'been', 'will', 'people', 'also', 'into', 'because', 'do', 'bad', 'other', 'great', 'how', 'first', 'him', 'most', 'dont', 'made', 'then', 'them', 'films', 'could', 'make', 'way', 'movies', 'after', 'too']


二，定义模型

使用Keras接口有以下3种方式构建模型：使用Sequential按层顺序构建模型，使用函数式API构建任意结构模型，继承Model基类构建自定义模型。


此处选择使用继承Model基类构建自定义模型。

In [31]:
# 演示自定义模型范例，实际上应该优先使用Sequential或者函数式API
tf.keras.backend.clear_session()

class CnnModel(models.Model):
    def __init__(self):
        super(CnnModel, self).__init__()
        
    def build(self,input_shape):
        self.embedding = layers.Embedding(MAX_WORDS,7,input_length=MAX_LEN)
        self.conv_1 = layers.Conv1D(16, kernel_size= 5,name = "conv_1",activation = "relu")
        self.pool_1 = layers.MaxPool1D(name = "pool_1")
        self.conv_2 = layers.Conv1D(128, kernel_size=2,name = "conv_2",activation = "relu")
        self.pool_2 = layers.MaxPool1D(name = "pool_2")
        self.flatten = layers.Flatten()
        self.dense = layers.Dense(1,activation = "sigmoid")
        super(CnnModel,self).build(input_shape)
    
    def call(self, x):
        x = self.embedding(x)
        x = self.conv_1(x)
        x = self.pool_1(x)
        x = self.conv_2(x)
        x = self.pool_2(x)
        x = self.flatten(x)
        x = self.dense(x)
        return(x)
    
    # 用于显示Output Shape
    def summary(self):
        x_input = layers.Input(shape = MAX_LEN)
        output = self.call(x_input)
        model = tf.keras.Model(inputs = x_input,outputs = output)
        model.summary()
    
model = CnnModel()
model.build(input_shape =(None,MAX_LEN))
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 200, 7)            70000     
_________________________________________________________________
conv_1 (Conv1D)              (None, 196, 16)           576       
_________________________________________________________________
pool_1 (MaxPooling1D)        (None, 98, 16)            0         
_________________________________________________________________
conv_2 (Conv1D)              (None, 97, 128)           4224      
_________________________________________________________________
pool_2 (MaxPooling1D)        (None, 48, 128)           0         
_________________________________________________________________
flatten (Flatten)            (None, 6144)             

三，训练模型

训练模型通常有3种方法，内置fit方法，内置train_on_batch方法，以及自定义训练循环。此处我们通过自定义训练循环训练模型。

In [32]:
#打印时间分割线
@tf.function
def printbar():
    today_ts = tf.timestamp()%(24*60*60)
    
    hour = tf.cast(today_ts//3600+8,tf.int32)%tf.constant(24)
    minite = tf.cast((today_ts%3600)//60,tf.int32)
    second = tf.cast(tf.floor(today_ts%60),tf.int32)
    
    def timeformat(m):
        if tf.strings.length(tf.strings.format("{}",m))==1:
            return(tf.strings.format("0{}",m))
        else:
            return(tf.strings.format("{}",m))
    
    timestring = tf.strings.join([timeformat(hour),timeformat(minite),
                timeformat(second)],separator = ":")
    tf.print("=========="*8+timestring)
optimizer = optimizers.Nadam()
loss_func = losses.BinaryCrossentropy()

train_loss = metrics.Mean(name='train_loss')
train_metric = metrics.BinaryAccuracy(name='train_accuracy')

valid_loss = metrics.Mean(name='valid_loss')
valid_metric = metrics.BinaryAccuracy(name='valid_accuracy')


@tf.function
def train_step(model, features, labels):
    with tf.GradientTape() as tape:
        predictions = model(features,training = True)
        loss = loss_func(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss.update_state(loss)
    train_metric.update_state(labels, predictions)
    

@tf.function
def valid_step(model, features, labels):
    predictions = model(features,training = False)
    batch_loss = loss_func(labels, predictions)
    valid_loss.update_state(batch_loss)
    valid_metric.update_state(labels, predictions)


def train_model(model,ds_train,ds_valid,epochs):
    for epoch in tf.range(1,epochs+1):
        
        for features, labels in ds_train:
            train_step(model,features,labels)

        for features, labels in ds_valid:
            valid_step(model,features,labels)
        
        #此处logs模板需要根据metric具体情况修改
        logs = 'Epoch={},Loss:{},Accuracy:{},Valid Loss:{},Valid Accuracy:{}' 
        
        if epoch%1==0:
            printbar()
            tf.print(tf.strings.format(logs,
            (epoch,train_loss.result(),train_metric.result(),valid_loss.result(),valid_metric.result())))
            tf.print("")
        
        train_loss.reset_states()
        valid_loss.reset_states()
        train_metric.reset_states()
        valid_metric.reset_states()

train_model(model,ds_train,ds_test,epochs = 6)

================================================================================00:27:12
Epoch=1,Loss:0.458000243,Accuracy:0.7519,Valid Loss:0.332691103,Valid Accuracy:0.8562

================================================================================00:27:24
Epoch=2,Loss:0.25274092,Accuracy:0.90055,Valid Loss:0.354468912,Valid Accuracy:0.858

================================================================================00:27:36
Epoch=3,Loss:0.181448966,Accuracy:0.932,Valid Loss:0.38288641,Valid Accuracy:0.8592

================================================================================00:27:49
Epoch=4,Loss:0.124764584,Accuracy:0.9543,Valid Loss:0.487780154,Valid Accuracy:0.8518

================================================================================00:28:01
Epoch=5,Loss:0.0811744258,Accuracy:0.97235,Valid Loss:0.5775792,Valid Accuracy:0.8554

================================================================================00:28:14
Epoch=6,Loss:0.0474837497,Accuracy

In [33]:
# 四，评估模型
# 通过自定义训练循环训练的模型没有经过编译，无法直接使用model.evaluate(ds_valid)方法


def evaluate_model(model,ds_valid):
    for features, labels in ds_valid:
         valid_step(model,features,labels)
    logs = 'Valid Loss:{},Valid Accuracy:{}' 
    tf.print(tf.strings.format(logs,(valid_loss.result(),valid_metric.result())))
    
    valid_loss.reset_states()
    train_metric.reset_states()
    valid_metric.reset_states()



In [34]:
evaluate_model(model,ds_test)

Valid Loss:0.781416059,Valid Accuracy:0.8466


五，使用模型

可以使用以下方法:
model.predict(ds_test)

model(x_test)

model.call(x_test)

model.predict_on_batch(x_test)

推荐优先使用model.predict(ds_test)方法，既可以对Dataset，也可以对Tensor使用。

In [35]:
model.predict(ds_test)

array([[0.21594143],
       [0.9999995 ],
       [0.9929304 ],
       ...,
       [0.9978466 ],
       [0.7512417 ],
       [1.        ]], dtype=float32)

In [36]:
for x_test,_ in ds_test.take(1):
    print(model(x_test))
    #以下方法等价：
    #print(model.call(x_test))
    #print(model.predict_on_batch(x_test))

tf.Tensor(
[[2.1594149e-01]
 [9.9999952e-01]
 [9.9293035e-01]
 [9.8558132e-07]
 [9.8469937e-01]
 [1.7200410e-03]
 [5.0301978e-06]
 [4.6303179e-05]
 [9.9996924e-01]
 [9.9994671e-01]
 [9.9997520e-01]
 [9.9848831e-01]
 [3.4775640e-06]
 [9.8042583e-01]
 [2.0674287e-10]
 [8.4212399e-01]
 [1.4600376e-05]
 [3.2137126e-02]
 [3.7461519e-04]
 [9.8414713e-01]], shape=(20, 1), dtype=float32)


六，保存模型
推荐使用TensorFlow原生方式保存模型。

In [37]:
model.save('./data/tf_model_savedmodel', save_format="tf")
print('export saved model.')

model_loaded = tf.keras.models.load_model('./data/tf_model_savedmodel')
model_loaded.predict(ds_test)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./data/tf_model_savedmodel/assets
export saved model.


array([[0.21594143],
       [0.9999995 ],
       [0.9929304 ],
       ...,
       [0.9978466 ],
       [0.7512417 ],
       [1.        ]], dtype=float32)